In [46]:
# Importy
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from typing import Callable

In [47]:
# Pobranie danych
heart_disease = fetch_ucirepo(id=45)

# Porzucenie linii z pustymi etykietami oraz odpowiadajacych im wartosci
feature_matrix = heart_disease.data.features.dropna()
labels = heart_disease.data.targets.loc[feature_matrix.index]

# Przetworzenie zbioru wartości przewidywanych do wartości binarnych
y_binary = labels.copy()
y_binary['num'] = y_binary['num'].apply(lambda x: 1 if x != 0 else 0)                      

# Utworznnie zbioru dummy etykiet
x_dummy = pd.get_dummies(feature_matrix, columns=['cp', 'restecg', 'slope','ca','thal'])         

# Podział danych na zbiór uczący i testowy
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y_binary, test_size=0.2, random_state=268555)

# Normalizacja cech trenignowych i testowych
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Konwersja danych do wymaganego formatu
x_train = np.array(x_train).astype(float)
y_train = np.array(y_train).astype(float)

In [48]:
# Funkcje aktywacji i ich pochodne
def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(0, x)

def relu_derivative(x: np.ndarray) -> np.ndarray:
    return (x > 0).astype(float)

def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x: np.ndarray) -> np.ndarray:
    s = sigmoid(x)
    return s * (1 - s)

In [49]:
# Funkcja kosztu i jej pochodna
def binary_cross_entropy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return -np.mean(y_true * np.log(y_pred + 1e-15) + (1 - y_true) * np.log(1 - y_pred + 1e-15))

def binary_cross_entropy_derivative(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    return (y_pred - y_true) / (y_pred * (1 - y_pred) + 1e-15)

In [50]:
# Klasa pojedynczej warstwy
class SingleLayer:
    def __init__(self,
                 input_layer_size : int,
                 output_layer_size : int,
                 activation : Callable[[np.ndarray, np.ndarray], float],
                 activation_derivative : Callable[[np.ndarray, np.ndarray], float],
                 std_dev : float = 0.01
                 )-> None:
        self.weights : np.ndarray = np.random.randn(input_layer_size, output_layer_size) * std_dev
        self.biases : np.ndarray = np.zeros((1, output_layer_size))
        self.activation : Callable[[np.ndarray, np.ndarray], float] = activation
        self.activation_derivative : Callable[[np.ndarray, np.ndarray], float] = activation_derivative
    
    def forward(self, feature_matrix: np.ndarray) -> np.ndarray:
        self.z = feature_matrix @ self.weights + self.biases
        self.a = self.activation(self.z)
        self.cache_x = feature_matrix
        return self.a

    def backward(self, gradient: np.ndarray) -> np.ndarray:
        delta = gradient * self.activation_derivative(self.z)
        self.d_weights = self.cache_x.T @ delta
        self.d_biases = np.sum(delta, axis=0, keepdims=True)
        return delta @ self.weights.T

    def update(self, learning_rate: float) -> None:
        self.weights -= learning_rate * self.d_weights
        self.biases -= learning_rate * self.d_biases

In [51]:
# Klasa sieci neuronowej
class NeuralNetwork:
    def __init__(self, layer_dims: list[int], std_dev: float = 0.01) -> None:
        self.layers: list[SingleLayer] = []
        for i in range(len(layer_dims) - 1):
            input_size: int = layer_dims[i]
            output_size: int = layer_dims[i + 1]
            if i < len(layer_dims) - 2:  # warstwy ukryte
                self.layers.append(SingleLayer(input_size, output_size, relu, relu_derivative, std_dev))
            else:  # ostatnia warstwa wyjściowa
                self.layers.append(SingleLayer(input_size, output_size, sigmoid, sigmoid_derivative, std_dev))

    def forward(self, feature_matrix: np.ndarray) -> np.ndarray:
        for layer in self.layers:
            feature_matrix = layer.forward(feature_matrix)
        return feature_matrix

    def backward(self, y_pred: np.ndarray, y_true: np.ndarray) -> None:
        gradient: np.ndarray = binary_cross_entropy_derivative(y_true, y_pred)
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)

    def update_weights(self, learning_rate: float) -> None:
        for layer in self.layers:
            layer.update(learning_rate)

    def train(self, feature_matrix: np.ndarray, labels: np.ndarray, epochs: int, learning_rate_param: float) -> None:
        global learning_rate
        learning_rate = learning_rate_param
        for epoch in range(epochs):
            y_pred = self.forward(feature_matrix)
            cost = binary_cross_entropy(labels, y_pred)
            self.backward(y_pred, labels)
            self.update_weights(learning_rate)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Cost: {cost}")

    def predict(self, X: np.ndarray) -> np.ndarray:
        return (self.forward(X) > 0.5).astype(int)

In [55]:
# Przykład użycia modelu
if __name__ == "__main__":
    # Ustawienia modelu
    layer_dims = [25, 8, 4, 2, 1]  # Przykładowe wymiary: input -> hidden -> output
    learning_rate = 0.01
    epochs = 1000

    # Tworzenie, trenowanie i testowanie modelu
    model = NeuralNetwork(layer_dims)
    model.train(x_train, y_train, epochs, learning_rate)

    # Sprawdzenie predykcji
    predictions = model.predict(x_test)
    accuracy = np.mean(predictions == y_test)
    print(f"Accuracy: {accuracy}")

Epoch 0, Cost: 0.6931471712791074
Epoch 100, Cost: 0.691642024463924
Epoch 200, Cost: 0.6916420225034614
Epoch 300, Cost: 0.6916420206971673
Epoch 400, Cost: 0.6916420187553498
Epoch 500, Cost: 0.6916420166666967
Epoch 600, Cost: 0.6916420143019294
Epoch 700, Cost: 0.6916420115271491
Epoch 800, Cost: 0.6916420083325854
Epoch 900, Cost: 0.6916420027700538
Accuracy: 0.5833333333333334
